<a href="https://colab.research.google.com/github/ss8319/Colab-Notebooks/blob/main/Shamus_Liquid_GAN_Simplified(youtube).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Liquid GAN (THE IMETATOR)

<a href="https://colab.research.google.com/drive/1bwUnj-9NnJA2EMr7eWO4I45UuBtKudg_?usp=sharing">Original Script</a>  ||  [![GitHub stars](https://img.shields.io/github/stars/iPERDance/iPERCore?style=social)](https://github.com/iPERDance/iPERCore)


In [ ]:
#@title <h1>Step1: Setup Liquid GAN</h1>
#@markdown * Install dependency
#@markdown * Download pretrained model

# Install ffmpeg (ffprobe)
!apt-get install ffmpeg

# set CUDA_HOME, here we use CUDA 10.1
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-10.1"
!echo $CUDA_HOME

!git clone https://github.com/iPERDance/iPERCore.git

%cd /content/iPERCore/
!python setup.py develop

# Download all checkpoints
#!wget -O assets/checkpoints.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLkwQyrk3W9_H7MzNA?e=rRje0G"
!wget -O assets/checkpoints.zip "https://download.impersonator.org/iper_plus_plus_latest_checkpoints.zip"
!unzip -o assets/checkpoints.zip -d assets/
!rm assets/checkpoints.zip

# download samples
# !wget -O assets/samples.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLobQPpoxo2hfhURrA?e=EUyIC2"
!wget -O assets/samples.zip  "https://download.impersonator.org/iper_plus_plus_latest_samples.zip"
!unzip -o assets/samples.zip -d  assets
!rm assets/samples.zip

!pip install -q youtube-dl
%cd /content/iPERCore/assets/
!git clone https://github.com/AwaleSajil/extra_liquid_gan_src_img

%cd /content/iPERCore/



import os
import os.path as osp
import platform
import argparse
import time
import sys
import subprocess
from IPython.display import HTML
from base64 import b64encode


# the gpu ids
gpu_ids = "0"

# the image size
image_size = 512

# the default number of source images, it will be updated if the actual number of sources <= num_source
num_source = 2

# the assets directory. This is very important, please download it from `one_drive_url` firstly.
assets_dir = "/content/iPERCore/assets"

# the output directory.
output_dir = "./results"

# the model id of this case. This is a random model name.
# model_id = "model_" + str(time.time())

# # This is a specific model name, and it will be used if you do not change it.
# model_id = "axing_1"

# symlink from the actual assets directory to this current directory
work_asserts_dir = os.path.join("./assets")
if not os.path.exists(work_asserts_dir):
    os.symlink(osp.abspath(assets_dir), osp.abspath(work_asserts_dir),
               target_is_directory=(platform.system() == "Windows"))

cfg_path = osp.join(work_asserts_dir, "configs", "deploy.toml")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/usr/local/cuda-10.1
Cloning into 'iPERCore'...
remote: Enumerating objects: 902, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 902 (delta 17), reused 29 (delta 14), pack-reused 864
Receiving objects: 100% (902/902), 23.05 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (334/334), done.
/content/iPERCore
Traceback (most recent call last):
  File "setup.py", line 357, in <module>
    torch_torchvision_mmcv = platform_dependencies()
  File "setup.py", line 226, in platform_dependencies
    cuda_version = get_cuda_version()
  File "setup.py", line 158, in get_cuda_version
 

In [ ]:
# @title <h1>Step 2: Inputs</h1>
import ipywidgets
from ipywidgets import interact, interactive, HBox, Label, VBox
from PIL import Image
import io
from IPython.display import clear_output
from glob import iglob
import ntpath


BASE_PATH = "/content/iPERCore/assets/"
SOURCE_BACKGROUND_IMAGE_PATH = BASE_PATH + "source_image_background.png"
SOURCE_IMAGE_DIR = BASE_PATH + "src_img"
!rm -rf $SOURCE_IMAGE_DIR
!mkdir $SOURCE_IMAGE_DIR
SOURCE_FRONT_IMAGE_PATH = SOURCE_IMAGE_DIR + '/' + "source_image_front.png"
SOURCE_BACK_IMAGE_PATH = SOURCE_IMAGE_DIR + '/' + "source_image_back.png"
POSE_FC = 300
CAM_FC = 150
VIDEO_TRIM_START = -1
VIDEO_TRIM_END = -1
INTERMEDIATE_VIDEO_PATH = BASE_PATH + "intVideo.mp4"
REFERECNE_VIDEOS = None
REFERECNE_VIDEOS_PATHS= None
MULTIANGLEIMG = False


def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail or ntpath.basename(head)


def referenceVideoList():
  global REFERECNE_VIDEOS, REFERECNE_VIDEOS_PATHS
  REFERECNE_VIDEOS = []
  REFERECNE_VIDEOS_PATHS = []

  for i, fn in enumerate(iglob('/content/iPERCore/assets/samples/references/*.mp4')):
    # print (path_leaf(fn))
    REFERECNE_VIDEOS_PATHS.append(fn)
    REFERECNE_VIDEOS.append((path_leaf(fn), i))
referenceVideoList()

def start_f(x):
  global VIDEO_TRIM_START
  VIDEO_TRIM_START = int(x)
  return x
def end_f(x):
  global VIDEO_TRIM_END
  VIDEO_TRIM_END = int(x)
  return x


def yt_1_f(x):
  #go download the video
  if(len(x)):
    #find youtube video id
    from urllib import parse as urlparse
    url_data = urlparse.urlparse(x)
    query = urlparse.parse_qs(url_data.query)
    YOUTUBE_ID = query["v"][0]

    !rm -df $INTERMEDIATE_VIDEO_PATH
    !rm -df youtube.mp4
    # download the youtube with the given ID
    !youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
    !mv youtube.mp4 $INTERMEDIATE_VIDEO_PATH
    clear_output()
    print("Downloaded")

  return x

def displayImage(path, w = 100, h= 100):
  file_ = open(path, "rb")
  image = file_.read()
  tmp = ipywidgets.Image(
      value=image,
      format='png',
      width=w,
      height=h,
  )
  display(tmp)

def demo_1_f(x): 
  demoList = [
              {"front": "/content/iPERCore/assets/samples/sources/afan_3/afan_3/IMG_7129.JPG",
               "back": "/content/iPERCore/assets/samples/sources/afan_3/afan_3/IMG_7131.JPG",
               "bg": "/content/iPERCore/assets/samples/sources/afan_3/IMG_7128.JPG"},
              
              {"front": "/content/iPERCore/assets/samples/sources/5NJIkUmh_h0/0001.PNG",
               "back": "",
               "bg": ""},
              
              {"front": "/content/iPERCore/assets/samples/sources/donald_trump_2/00000.PNG",
               "back": "",
               "bg": ""},
              
              {"front": "/content/iPERCore/assets/extra_liquid_gan_src_img/Vladimir_Putin.jpg",
               "back": "",
               "bg": ""},
              {"front": "/content/iPERCore/assets/extra_liquid_gan_src_img/modi.png",
               "back": "",
               "bg": ""},
              {"front": "/content/iPERCore/assets/extra_liquid_gan_src_img/queen_elizabeth.jpg",
               "back": "",
               "bg": ""},
              {"front": "/content/iPERCore/assets/extra_liquid_gan_src_img/zemo1.jpg",
               "back": "",
               "bg": ""},
              {"front": "/content/iPERCore/assets/extra_liquid_gan_src_img/zemo2.jpg",
               "back": "",
               "bg": ""},
              {"front": "/content/iPERCore/assets/extra_liquid_gan_src_img/zemo3.jpg",
               "back": "",
               "bg": ""},
              
  ]

  paths = demoList[int(x)]
  !rm -f $SOURCE_BACKGROUND_IMAGE_PATH
  !rm -f $SOURCE_FRONT_IMAGE_PATH
  !rm -f $SOURCE_BACK_IMAGE_PATH

  if (paths["front"] != ""):
    img = Image.open(paths["front"])
    img.save(SOURCE_FRONT_IMAGE_PATH)

  if (paths["back"] != ""):
    img = Image.open(paths["back"])
    img.save(SOURCE_BACK_IMAGE_PATH)

  if (paths["bg"] != ""):
    img = Image.open(paths["bg"])
    img.save(SOURCE_BACKGROUND_IMAGE_PATH)

  return x

def upload_1_f(x):
  return upload_img_f(x,y = SOURCE_FRONT_IMAGE_PATH) 
def upload_2_f(x):
  return upload_img_f(x,y = SOURCE_BACK_IMAGE_PATH) 
def upload_3_f(x):
  return upload_img_f(x,y = SOURCE_BACKGROUND_IMAGE_PATH) 

def upload_img_f(x,y):
  global MULTIANGLEIMG
  clear_output()

  path_to_save_img = y
  if (len(list(x.keys())) == 1):
    MULTIANGLEIMG = False
    file = x[list(x.keys())[0]]['content']
    img = Image.open(io.BytesIO(x[list(x.keys())[0]]['content']))
    !rm -f $path_to_save_img
    img.save(path_to_save_img)
    print("Uploaded: " + str(list(x.keys())[0]))
    displayImage(path_to_save_img) 
  elif (len(list(x.keys())) > 1):
    MULTIANGLEIMG = True
    !rm -rf $SOURCE_IMAGE_DIR
    !mkdir $SOURCE_IMAGE_DIR
    for i in range(len(list(x.keys()))):
      file = x[list(x.keys())[i]]['content']
      img = Image.open(io.BytesIO(x[list(x.keys())[i]]['content']))
      file_namei = SOURCE_IMAGE_DIR + '/srcimg' + str(i) + '.png'
      img.save(file_namei)
      print("Uploaded: " + str(list(x.keys())[i]))
  return x

def upload_video_f(x):
  if (len(list(x.keys()))):
    videoUploadButton.style.button_color = "yellow"
    file = x[list(x.keys())[0]]['content']
    !rm -f temp
    with open("./temp", "wb") as fp:
      fp.write(file)
    !rm -f $INTERMEDIATE_VIDEO_PATH
    !ffmpeg -i "temp" $INTERMEDIATE_VIDEO_PATH
    clear_output()
    print("Downloaded: " + str(list(x.keys())[0]))
    videoUploadButton.style.button_color = "lightgreen"
  return x

def web_1_f(x):
  return web_img_f(x,y = SOURCE_FRONT_IMAGE_PATH) 
def web_2_f(x):
  return web_img_f(x,y = SOURCE_BACK_IMAGE_PATH) 
def web_3_f(x):
  return web_img_f(x,y = SOURCE_BACKGROUND_IMAGE_PATH) 

def web_img_f(x,y):
  path_to_save_img = y
  if (len(x)):
    !rm -f "temp"
    try:
      downloadState = !wget -O "temp" $x
      downloadState = ''.join(downloadState)
      if ("unable to resolve host address" in downloadState):
        print("URL Invalid")
      else:
        img = Image.open("temp")
        !rm -f $path_to_save_img
        img.save(path_to_save_img)
        print("Downloaded")
        displayImage(path_to_save_img)
    except:
      print("Invalid Image URL")
  return x

def web_video_1_f(x):
  # INTERMEDIATE_VIDEO_PATH
  if (len(x)):
    !rm -f "temp"
    try:
      downloadState = !wget -O "temp" $x
      downloadState = ''.join(downloadState)
      if ("unable to resolve host address" in downloadState):
        print("URL Invalid")
      else:
        !ffmpeg -i "temp" $INTERMEDIATE_VIDEO_PATH
        clear_output()
        print("Downloaded")
    except:
      print("Invalid Image URL")
  
def demo_vid_f(x):
  ref_vid_path = REFERECNE_VIDEOS_PATHS[int(x)]
  !rm -f $INTERMEDIATE_VIDEO_PATH
  !cp $ref_vid_path $INTERMEDIATE_VIDEO_PATH

  return x
def pose_fc_f(x):
  global POSE_FC
  POSE_FC = str(x)
  return x
def cam_fc_f(x):
  global CAM_FC
  CAM_FC = str(x)
  return x

instructions = ipywidgets.widgets.HTML('<h2>Source Image</h2>')
display(instructions)


# --------------------------------------------------------------------------------------
#------------------------------first tab------------------------------------------------
demo_1 = HBox([Label('*Choose source image:'), interactive(demo_1_f, x = ipywidgets.Dropdown(
    options=[('Man', 0), ('Women', 1), ('Donald Trump', 2), 
             ('Vladimir Putin', 3), ('Narendra Modi', 4), ('Queen Elizabeth', 5),
             ('Zemo(MCU)', 6), ('Zemo(MCU)', 6), ('Zemo(MCU)', 6)],
    value=2,
    description=""
))])

# --------------------------------------------------------------------------------------
#------------------------------second tab-----------------------------------------------
upload_1 = HBox([Label('*Upload Front facing image:'), interactive(upload_1_f, x = ipywidgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
    
))])
upload_2 = HBox([Label('Upload Back facing image:'), interactive(upload_2_f, x = ipywidgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    
))])
upload_3 = HBox([Label('Upload background image:'), interactive(upload_3_f, x = ipywidgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    
))])

upload_tab = ipywidgets.VBox([upload_1, upload_2, upload_3])


# --------------------------------------------------------------------------------------
#------------------------------third tab------------------------------------------------
web1 = HBox([Label('*Front facing image link:'), interactive(web_1_f, x = ipywidgets.Text(
    value='',
    placeholder='Type url for image',
    description='',
    disabled=False
))])
web2 = HBox([Label('Back facing image link:'), interactive(web_2_f, x = ipywidgets.Text(
    value='',
    placeholder='Type url for image',
    description='',
    disabled=False
))])
web3 = HBox([Label('Background image link:'), interactive(web_3_f, x = ipywidgets.Text(
    value='',
    placeholder='Type url for image',
    description='',
    disabled=False
))])
web_tab = ipywidgets.VBox([web1, web2, web3])



source_img_tabs = ipywidgets.Tab()
source_img_tabs.children = [demo_1, upload_tab, web_tab]
source_img_tabs.set_title(0, 'Demo')
source_img_tabs.set_title(1, 'Upload from pc')
source_img_tabs.set_title(2, 'From web')
display(source_img_tabs)


instructions = ipywidgets.widgets.HTML('<h2>Reference/Driving Video</h2>')
display(instructions)


# --------------------------------------------------------------------------------------
#------------------------------first tab------------------------------------------------
demo_1 = HBox([Label('*Choose driving video:'), interactive(demo_vid_f, x = ipywidgets.Dropdown(
    options=REFERECNE_VIDEOS,
    value=0,
    description=""
))])

start_1 = HBox([Label('Start Time (second):'), interactive(start_f, x = ipywidgets.IntText(
    value=-1,
    description="",
    disabled=False
))])

end_1 = HBox([Label('End Time (second):'), interactive(end_f, x = ipywidgets.IntText(
    value=-1,
    description="",
    disabled=False
))])

pose_fc_ = HBox([Label('Pose_fc:'), interactive(pose_fc_f, x = ipywidgets.IntText(
    value=300,
    description="   The 'pose_fc' is the smooth factor of the temporal poses. The smaller of this value, the smoother of the temporal poses. If it is empty, we will set it as the default 300. In the most cases, using the default 300 is enough, and if you find the poses of the outputs are not stable, you can decrease this value. Otherwise, if you find the poses of the outputs are over stable, you can increase this value.",
    disabled=False
))])

cam_fc_ = HBox([Label('Cam_fc:'), interactive(cam_fc_f, x = ipywidgets.IntText(
    value=150,
    description="   The 'cam_fc' is the smooth factor of the temporal cameras (locations in the image space). The smaller of this value, the smoother of the locations in sequences. If it is empty, we will set it as the default 150. In the most cases, the default 150 is enough.",
    disabled=False
))])


demo_tab = ipywidgets.VBox([demo_1, start_1, end_1,pose_fc_, cam_fc_])


# --------------------------------------------------------------------------------------
#------------------------------second tab-----------------------------------------------
yt1 = HBox([Label('*Driving youtube link:'), interactive(yt_1_f, x = ipywidgets.Text(
    value='',
    placeholder='Type youtube url link',
    description='',
    disabled=False
))])

yt_tab = ipywidgets.VBox([yt1,start_1, end_1,pose_fc_, cam_fc_])
# --------------------------------------------------------------------------------------
#------------------------------third tab-----------------------------------------------
videoUploadButton = ipywidgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    
)
upload_1 = HBox([Label('*Upload driving video:'), interactive(upload_video_f, x = videoUploadButton)])

upload_tab = ipywidgets.VBox([upload_1, start_1, end_1, pose_fc_, cam_fc_])



# --------------------------------------------------------------------------------------
#------------------------------fourth tab------------------------------------------------
web1 = HBox([Label('*Driving video link:'), interactive(web_video_1_f, x = ipywidgets.Text(
    value='',
    placeholder='Type url for video',
    description='',
    disabled=False
))])

web_tab = ipywidgets.VBox([web1,start_1, end_1, pose_fc_, cam_fc_])



driving_tabs = ipywidgets.Tab()
driving_tabs.children = [demo_tab, yt_tab,upload_tab, web_tab]
driving_tabs.set_title(0, 'Demo')
driving_tabs.set_title(1, 'From Youtube')
driving_tabs.set_title(2, 'Upload from pc')
driving_tabs.set_title(3, 'From web')
display(driving_tabs)

HTML(value='<h2>Source Image</h2>')

HTML(value='<h2>Reference/Driving Video</h2>')

In [ ]:
# @title <h1>Step 3: Process + Output</h1>

#first process referecence viddeo
TRIMMED_REFERECNE_VIDEO_PATH = BASE_PATH +  "referVideo.mp4"
# trim the video 

# VIDEO_TRIM_START = -1
# VIDEO_TRIM_END = -1
if VIDEO_TRIM_START == -1 or VIDEO_TRIM_END == -1:
  # just copy the intVideo as referce video
  !rm -f $TRIMMED_REFERECNE_VIDEO_PATH
  !cp $INTERMEDIATE_VIDEO_PATH $TRIMMED_REFERECNE_VIDEO_PATH
else:
  interval = VIDEO_TRIM_END - VIDEO_TRIM_START
  !ffmpeg -y -i $INTERMEDIATE_VIDEO_PATH -ss {VIDEO_TRIM_START} -t {interval} -async 1 $TRIMMED_REFERECNE_VIDEO_PATH

clear_output()


#generate source path
import os.path
from os import path


s_path = ""
s_name = ""
s_bg = ""

if path.exists(SOURCE_FRONT_IMAGE_PATH):
  s_path = "path?=" + SOURCE_FRONT_IMAGE_PATH
  s_name = ",name?=" + (path_leaf(SOURCE_FRONT_IMAGE_PATH)).split(".")[0]



nFilesSrc = !find $SOURCE_IMAGE_DIR -type f | wc -l
nFilesSrc = int(nFilesSrc[0])

if nFilesSrc > 1:
  s_path = "path?=" + SOURCE_IMAGE_DIR
  s_name = ",name?=" + path_leaf(SOURCE_IMAGE_DIR)

if (path.exists(SOURCE_BACK_IMAGE_PATH) and path.exists(SOURCE_FRONT_IMAGE_PATH)):
  s_path = "path?=" + SOURCE_IMAGE_DIR
  s_name = ",name?=" + path_leaf(SOURCE_IMAGE_DIR)

if (path.exists(SOURCE_BACKGROUND_IMAGE_PATH)):
  s_bg = ",bg_path?=" + SOURCE_BACKGROUND_IMAGE_PATH


src_path = "\"" + s_path + s_name + s_bg + "\""



r_path = "path?=" + TRIMMED_REFERECNE_VIDEO_PATH
r_name = ",name?=" + (path_leaf(TRIMMED_REFERECNE_VIDEO_PATH)).split(".")[0]
r_pose_fc = ",pose_fc?=" + str(POSE_FC)
r_cam_fc = ",cam_fc?=" + str(CAM_FC)


ref_path = "\"" + r_path + r_name + r_pose_fc + r_cam_fc + "\""



# This is a specific model name, and it will be used if you do not change it. This is the case of `trump`
model_id = "Custom"

try: 
  !rm -rf /content/iPERCore/results
  !python -m iPERCore.services.run_imitator  \
    --gpu_ids     $gpu_ids       \
    --num_source  $num_source    \
    --image_size  $image_size    \
    --output_dir  $output_dir    \
    --model_id    $model_id      \
    --cfg_path    $cfg_path      \
    --src_path    $src_path      \
    --ref_path    $ref_path
  
  clear_output()
except:
  print("Something isn't right")


p1 = "/content/iPERCore/results/primitives/source_image_front/synthesis/imitations/source_image_front-referVideo.mp4"
p2 = "/content/iPERCore/results/primitives/src_img/synthesis/imitations/src_img-referVideo.mp4"
def displayVideo(p):
  mp4 = open(p, "rb").read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  x = HTML(f"""
  <video width="100%" height="100%" controls>
        <source src="{data_url}" type="video/mp4">
  </video>""")
  return x

if path.exists(p1):
  display(displayVideo(p1))
elif path.exists(p2):
  display(displayVideo(p2))
else:
  print("No result Found")